In [0]:
import json
import re

def convert_tmpl_to_ipynb(tmpl_file_path, ipynb_file_path):
    # Read the .tmpl file
    with open(tmpl_file_path, 'r') as tmpl_file:
        tmpl_content = tmpl_file.read()

    # Convert Jinja template to Python code
    # Step 1: Replace Jinja templating with dynamic values (for now, we'll just replace with placeholders)
    tmpl_content = re.sub(r'{{\s*template\s`(.*?)`\s.*?}}', r'{{ \1 }}', tmpl_content)  # Handle Jinja template expressions
    tmpl_content = re.sub(r'{{\s*if\s(.*?)\s}}', r'if \1:', tmpl_content)  # Handle Jinja if conditions
    tmpl_content = re.sub(r'{{\s*else\s}}', r'else:', tmpl_content)  # Handle Jinja else
    tmpl_content = re.sub(r'{{\s*end\s-?}}', r'', tmpl_content)  # Handle Jinja end

    # Step 2: Create Jupyter notebook structure (JSON format)
    notebook = {
        "cells": [],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            },
            "language_info": {
                "codemirror_mode": {
                    "name": "ipython",
                    "version": 3
                },
                "file_extension": ".py",
                "mimetype": "text/x-python",
                "name": "python",
                "nbconvert_exporter": "python",
                "version": "3.8.5"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 5
    }

    # Step 3: Split the file into markdown and code blocks based on notebook sections
    sections = tmpl_content.split("# COMMAND ----------")
    for section in sections:
        if section.startswith("# DBTITLE") or section.startswith("# MAGIC"):
            # It's a markdown section (if it starts with "# DBTITLE" or "# MAGIC")
            notebook['cells'].append({
                "cell_type": "markdown",
                "metadata": {},
                "source": [section.replace("# MAGIC ", "").strip()]
            })
        else:
            # Otherwise, it's a code section
            notebook['cells'].append({
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "outputs": [],
                "source": [section.strip()]
            })

    # Step 4: Write the notebook as a .ipynb file
    with open(ipynb_file_path, 'w') as ipynb_file:
        json.dump(notebook, ipynb_file, indent=4)

    print(f"Notebook has been saved as {ipynb_file_path}")

# Example usage:
tmpl_file_path = 'Train.py.tmpl'  # Path to your .tmpl file
ipynb_file_path = 'train_test.ipynb'  # Path where the .ipynb will be saved

convert_tmpl_to_ipynb(tmpl_file_path, ipynb_file_path)